In [10]:
# --- 1. Imports ---
import pandas as pd
import numpy as np
import re
import joblib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.pipeline import Pipeline

# Import NLTK components from your cleaning file (optional)
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
# --- 2. Load the cleaned dataset ---
print("📂 Loading cleaned dataset...")
df = pd.read_csv("cleaned_spam_dataset.csv")
print(f"✅ Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print("Label distribution:\n", df['Label'].value_counts())

📂 Loading cleaned dataset...
✅ Dataset loaded: 28397 rows, 3 columns
Label distribution:
 Label
0    19821
1     8576
Name: count, dtype: int64


In [12]:
# --- 3. Split data ---
X = df['Body']
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"🧩 Training size: {X_train.shape[0]} | Testing size: {X_test.shape[0]}")


🧩 Training size: 22717 | Testing size: 5680


In [13]:
# --- 4. TF-IDF Vectorization ---
tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words='english'
)

In [14]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


In [15]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)
print("✅ Model training complete!")

✅ Model training complete!


In [16]:
y_pred = model.predict(X_test_tfidf)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, digits=4))
print("\n📈 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\n⭐ F1 Macro Score:", round(f1_score(y_test, y_pred, average='macro'), 4))



📊 Classification Report:
              precision    recall  f1-score   support

           0     0.9627    0.9841    0.9733      3965
           1     0.9613    0.9120    0.9360      1715

    accuracy                         0.9623      5680
   macro avg     0.9620    0.9480    0.9546      5680
weighted avg     0.9623    0.9623    0.9620      5680


📈 Confusion Matrix:
[[3902   63]
 [ 151 1564]]

⭐ F1 Macro Score: 0.9546


In [17]:
# --- 7. Save Model and Vectorizer ---
joblib.dump(model, "spam_classifier_model.pkl")
joblib.dump(tfidf, "tfidf_vectorizer.pkl")
print("💾 Model and TF-IDF vectorizer saved successfully!")

💾 Model and TF-IDF vectorizer saved successfully!


In [20]:
# --- 8. Utility: Clean Text Function (same as cleaning script) ---
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return ' '.join(words)


In [26]:
# --- 9. Inference Function ---
def predict_email(text):
    cleaned = clean_text(text)
    features = tfidf.transform([cleaned])
    prob = model.predict_proba(features)[0]   # probabilities for both classes
    pred = np.argmax(prob)
    confidence = round(prob[pred] * 100, 2)
    label = "Spam" if pred == 1 else "Ham"
    return label, confidence

In [27]:
# --- 10. Quick Test Prediction ---
label, conf = predict_email("Win a brand new iPhone!")
print(f"Prediction: {label} ({conf}% confidence)")


Prediction: Spam (79.78% confidence)


In [23]:
# --- 11. Full Pipeline Option (TF-IDF + Model together) ---
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english')),
    ('clf', LogisticRegression(max_iter=1000))
])

In [24]:
pipeline.fit(X_train, y_train)
pipeline_score = pipeline.score(X_test, y_test)
print(f"\n📦 Combined Pipeline Accuracy: {pipeline_score:.4f}")


📦 Combined Pipeline Accuracy: 0.9620


In [25]:
joblib.dump(pipeline, "spam_pipeline.pkl")
print("✅ Combined pipeline saved as spam_pipeline.pkl")
print("\n🎯 Training complete — ready for deployment!")

✅ Combined pipeline saved as spam_pipeline.pkl

🎯 Training complete — ready for deployment!
